In [1]:
import osmium
import os
from itertools import islice

In [2]:
node_usage = {}

In [3]:
class NodeIndexer(osmium.SimpleHandler):
    def node(self, n: osmium.Node):
        node_usage[n.id] = []

    def way(self, w: osmium.Way):
        if 'highway' not in w.tags:
            return
        for i in w.nodes:
            node_usage[i.ref].append(w.id)

In [4]:
node_usage = {}
handler = NodeIndexer()
handler.apply_file('../osm/export.osm')

In [5]:
class WayDivider(osmium.SimpleHandler):
    writer: osmium.SimpleWriter
    
    def __init__(self, writer: osmium.SimpleWriter):
        super().__init__()
        self.writer = writer
        self.new_id_ = 1

    def new_id(self):
        a = self.new_id_
        self.new_id_ += 1
        return a
        
    def node(self, n: osmium.Node):
        self.writer.add_node(n)

    def way(self, w: osmium.Way):
        if 'highway' not in w.tags:
            return
        divided_to = 0
        divided = [w.nodes[0]]
        for i in islice(w.nodes, 1, None):
            divided.append(i)
            if len(node_usage[i.ref]) > 1:
                self.writer.add_way(w.replace(nodes=divided, id=self.new_id()))
                divided = [i]
                divided_to += 1
        if len(divided) > 1:
            self.writer.add_way(w.replace(nodes=divided, id=self.new_id()))
            divided_to += 1
        if divided_to == 0:
            raise Exception(f"divided to 0: {w.id}")
        if divided_to < 2 and (w.nodes.ends_have_same_id() or w.is_closed()):
            print(divided_to)
            raise Exception(f"w.nodes.ends_have_same_id() or w.is_closed(): {w.id}")

In [6]:
try:
    os.remove("../osm/divided.osm")
except FileNotFoundError:
    pass
writer = osmium.SimpleWriter('../osm/divided.osm')
handler = WayDivider(writer)
handler.apply_file('../osm/export.osm')
writer.close()